In [1]:
import os
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document


In [ ]:
import dotenv
dotenv.load_dotenv()

base_path = os.getenv("BASE_PATH", ".")
folder_path = os.path.join(base_path,"html_files")

for idx, file_name in enumerate(os.listdir(folder_path)):
    if file_name.endswith(".html"):
        file_path = os.path.join(folder_path, file_name)
        loader = UnstructuredHTMLLoader(file_path)
        documents = loader.load()
        if len(file_name.split('_')) != 6:
            print(file_name,)
        _,_, bc, bt, st, _ = file_name.split('_')
        
        
        for doc in documents:
            remove_first_line = '\n'.join(doc.page_content.split('\n')[1:])
            final_content = f"과목명: {bc}"
            final_content += f"주제: {bt}"
            final_content += f"소주제: {st}"
            final_content += remove_first_line
            doc.page_content = final_content
            doc.metadata['file_name'] = file_name
            doc.metadata['chapter'] = bc
            doc.metadata['big_title'] = bt
            doc.metadata['small_title'] = st
        
        if idx == 0:
            all_documents = documents
        else:
            all_documents.extend(documents)


In [26]:
for i in all_documents:
    if len(i.page_content) >500:
        print(i.metadata['source'])
        print(i.page_content)
        print('='*100)

C:\Users\Sese\autosave\meditalk\html_files\html_file_-2. 전공의 산부인과_1. 임산부의 영상검사_1. 임산부의 영상검사_1341.html
전공의 산부인과###임산부의 영상검사

(2848, '임산부의 영상검사 ', 'basic', 1)

초음파와 MRI: 초음파와 MRI와 관련된 태아 위해는 보고된 바 없다. 따라서 임산부에서 가장 먼저 고려하는 영상검사이다.

조영제: 임산부에 있어 가돌리늄 조영제는 꼭 필요한 경우에만 사용되어야 한다.

가돌리늄 MRI 와 MRI 촬영이 없었던 임신을 비교했을 때, 류마티스, 염증성 및 피부질환을 앓은 신생아가 유의하게 더 많게 보고된 코호트 연구가 있어, 위해와 이익을 충분히 비교해본 후에 시행할 것을 권고한다.

X ray, CT: 방사선 피폭에 대한 우려로 방사선 검사는 되도록 피하는 것이 좋으나, 의학적으로 꼭 필요한 경우엔 시행되어야 한다. 그러나 항상 초음파 및 MRI를 우선적으로 고려해야 한다.

취약 시기: 방사선으로 인한 중추신경계 기형(e.g> 지적장애 등)은 태아 주수 8~15주경에 가장 취약하다고 알려져 있다.

일반적인 teratogen(기형유발물질)의 경우 3~8주경이 가장 취약한 시기이다.

영향: 임신 중 50mGy 이하의 방사선 노출은 태아의 기형이나 유산 확률을 증가시키지 않는다. 임신 중절은 여러 인자의 영향을 받는 개인적인 의사결정이나 100mGy 이하의 태아 선량에 대해선 임신 중절을 고려하는 것이 적절치 않다.



Chest PA의 경우 0.01mGy 미만이며, 대부분의 진단적 목적은 50mGy 에 한참 못 미친다.

물론, 방사선 검사는 최대한 피하는 것이 좋지만, 우연히 임신한 줄 모르고 X-ray를 촬영한 산모가 주어졌을 때, 1회의 진단 목적의 X-ray 검사는 태아에게 해로운 결과를 초래하지 않으므로 이로 인해 임신 중절을 고려하는 건 적절치 않다. 경과관찰, 혹은 해당 주수에 맞는 산전진찰을 시행한다.

방사선 동위원소: 임신 

In [ ]:

embedding = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
)
vectorstore = Chroma(
    collection_name="html_docs",
    embedding_function=embedding,
    persist_directory="chroma_db",
)

